In [2]:
import wrds
import pandas as pd
from scipy.optimize import fsolve
from scipy.stats import norm
import numpy as np


# WRDS 데이터베이스 연결
# db = wrds.Connection()

# compustat_data = db.raw_sql("""
#     SELECT gvkey, cusip, datadate, at, act, lct, che, rect, invt, cogs, sale, ni, ib, dltt, dlc, oiadp, dp
#     FROM comp.funda
#     WHERE datadate >= '1990-01-01';
# """)

# crsp_data = db.raw_sql("""
#     SELECT permno, cusip, date, prc, shrout
#     FROM crsp.msf
#     WHERE date >= '1990-01-01' AND date <= '2024-12-31'
# """)

# 날짜 형식을 datetime으로 변환하고 연도 추출

crsp_data = pd.read_csv('crsp_data.csv')
compustat_data = pd.read_csv('compustat_data.csv')
rf_data = pd.read_csv('rf_info.csv')

crsp_data['date'] = pd.to_datetime(crsp_data['date'])
crsp_data['year'] = crsp_data['date'].dt.year
crsp_data['month'] = crsp_data['date'].dt.month


compustat_data['datadate'] = pd.to_datetime(compustat_data['datadate'])
compustat_data['year'] = compustat_data['datadate'].dt.year

rf_data['date'] = pd.to_datetime(rf_data['dateff'])
rf_data['year'] = rf_data['date'].dt.year
rf_data['month'] = rf_data['date'].dt.month


# Distance-to-Default (DD) 계산 함수
def calculate_dd(row, sigma_E, rf_avg=0.03, T=1):
    V_E = row['prc'] * row['shrout']  # 시가총액 계산
    D = row['dltt'] + row['dlc']  # 총 부채 (장기 부채 + 단기 부채)

    # 부채(D) 값이 0이거나 음수인 경우 처리
    if D <= 0:
        return np.nan  # 부채가 0이거나 음수이면 DD 계산 불가능
    
    # 블랙-숄즈 방정식에 필요한 자산 가치 및 변동성 추정
    def equations(p):
        V_A, sigma_A = p
        if sigma_A <= 0:  # sigma_A가 0 이하인 경우 처리
            return [np.nan, np.nan]

        d1 = (np.log(V_A / D) + (rf_avg + 0.5 * sigma_A ** 2) * T) / (sigma_A * np.sqrt(T))
        d2 = d1 - sigma_A * np.sqrt(T)
        eq1 = V_E - (V_A * norm.cdf(d1) - D * np.exp(-rf_avg * T) * norm.cdf(d2))
        eq2 = sigma_E - (norm.cdf(d1) * (V_A / V_E) * sigma_A)
        return [eq1, eq2]
    
    # 초기 추정값 설정 (자산 가치와 변동성의 초기값)
    initial_guess = [V_E + D, sigma_E]
    
    try:
        V_A, sigma_A = fsolve(equations, initial_guess)
    except Exception as e:
        return np.nan  # 계산 실패 시 NaN 반환
    
    if V_A <= 0 or sigma_A <= 0:  # 자산 가치 또는 변동성 값이 음수인 경우 처리
        return np.nan
    
    # Distance-to-Default 계산
    d1 = (np.log(V_A / D) + (rf_avg + 0.5 * sigma_A ** 2) * T) / (sigma_A * np.sqrt(T))
    DD = (np.log(V_A / D) + (rf_avg - 0.5 * sigma_A ** 2) * T) / (sigma_A * np.sqrt(T))
    return DD

# Step 1: Compustat 데이터와 CRSP 데이터를 연도별로 병합
# CRSP는 월별 데이터, Compustat은 연도별 데이터이므로 연도를 기준으로 병합

merged_december_data = pd.merge(crsp_data, rf_data[['year', 'month', 'rf']], on=['year', 'month'], how='left')


merged_data = pd.merge(merged_december_data, compustat_data, on=['cusip', 'year'], how='left')

# Step 2: CUSIP과 연도별로 그룹화하여 주가 변동성 계산 (1년 1~12월 표준 편차)
volatility_data = merged_data.groupby(['cusip', 'year']).agg(
    sigma_E=('prc', lambda x: np.std(x) if len(x) == 12 else np.nan)  # 12개월 주가 변동성만 계산
).reset_index()

# Step 3: 12월 데이터 필터링
december_data = merged_data[merged_data['month'] == 12]

# Step 4: 변동성 데이터와 12월 데이터를 병합
december_data = pd.merge(december_data, volatility_data, on=['cusip', 'year'], how='left')

# Step 5: DD 계산
# DD를 계산하기 위해 각 행에서 Compustat의 부채 데이터(dltt, dlc)를 사용
december_data['DD'] = december_data.apply(lambda row: calculate_dd(row, row['sigma_E'],row['rf']), axis=1)
december_data.dropna().to_csv('merged_data.csv', index=False)

# db.close()



/var/folders/px/2gqqh24s6yb9zp3q9m6th5f80000gn/T/ipykernel_24236/2868321701.py:57: RuntimeWarning: invalid value encountered in log
  d1 = (np.log(V_A / D) + (rf_avg + 0.5 * sigma_A ** 2) * T) / (sigma_A * np.sqrt(T))
/var/folders/px/2gqqh24s6yb9zp3q9m6th5f80000gn/T/ipykernel_24236/2868321701.py:67: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  V_A, sigma_A = fsolve(equations, initial_guess)


KeyboardInterrupt: 

In [3]:
december_data

,permno,cusip,date,prc,shrout,year,month,rf,gvkey,datadate,...,cogs,sale,ni,ib,dltt,dlc,oiadp,dp,sigma_E,DD
0,10001,36720410,1990-12-31,-9.50000,1054.0,1990,12,0.0060,12994.0,1990-06-30,...,19.907,23.227,1.131,1.131,7.175,0.195,2.459,0.861,9.608531,NaN
1,10002,05978R10,1990-12-31,-5.00000,1175.0,1990,12,0.0060,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.511478,NaN
2,10003,39031810,1990-12-31,2.50000,2194.0,1990,12,0.0060,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.085624,NaN
3,10005,95815510,1990-12-31,-0.04688,8347.0,1990,12,0.0060,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.009298,NaN
4,10009,46334710,1990-12-31,6.75000,941.0,1990,12,0.0060,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.989674,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263652,93426,92835K10,2023-12-29,34.07000,12510.0,2023,12,0.0043,185138.0,2023-12-31,...,188.821,355.048,25.707,25.707,54.481,4.004,43.849,15.550,4.295178,-0.075621
263653,93427,G3323L10,2023-12-29,190.33000,36331.0,2023,12,0.0043,179583.0,2023-06-30,...,2265.356,2645.237,247.913,247.913,0.066,13.357,258.600,43.832,26.351835,-12.676654
263654,93429,12503M10,2023-12-29,178.56000,105556.0,2023,12,0.0043,184500.0,2023-12-31,...,1855.500,3773.500,761.400,761.400,1590.000,20.800,1050.900,158.000,18.687229,-8.842110
263655,93434,78513510,2023-12-29,0.70000,43039.0,2023,12,0.0043,184259.0,2023-06-30,...,58.983,73.521,14.410,14.410,6.449,50.046,-16.659,4.769,0.345118,18.073490
